# E-commerce customer segmentation - project

Objective is to analyse raw data from an E-commerce application and segment customers based on their buying behaviour using RFM-analysis. 

Dataset used can be found here: https://archive.ics.uci.edu/dataset/502/online+retail+ii


## Data Cleaning

In [13]:
import pandas as pd
import numpy as np

# Importing the dataset from an excel file
df = pd.read_excel('../data/raw/online_retail_II.xlsx')

# Displaying the first few rows of the dataset
df.head(3)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom


In [14]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,525461.000000,525461,525461.000000,417534.000000
mean,10.337667,2010-06-28 11:37:36.845017856,4.688834,15360.645478
min,-9600.000000,2009-12-01 07:45:00,-53594.360000,12346.000000
25%,1.000000,2010-03-21 12:20:00,1.250000,13983.000000
50%,3.000000,2010-07-06 09:51:00,2.100000,15311.000000
75%,10.000000,2010-10-15 12:45:00,4.210000,16799.000000
max,19152.000000,2010-12-09 20:01:00,25111.090000,18287.000000
std,107.424110,NaN,146.126914,1680.811316


In [ ]:
# Identify missing rows where customer ID is missing
missing_customer_id = df[df['Customer ID'].isnull()]
missing_customer_id.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107927 entries, 263 to 525235
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      107927 non-null  object        
 1   StockCode    107927 non-null  object        
 2   Description  104999 non-null  object        
 3   Quantity     107927 non-null  int64         
 4   InvoiceDate  107927 non-null  datetime64[ns]
 5   Price        107927 non-null  float64       
 6   Customer ID  0 non-null       float64       
 7   Country      107927 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 7.4+ MB


In [16]:
# Dataframe without missing customerIDs
df1 = df.copy()
df1.dropna(subset=['Customer ID'], inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      417534 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 28.7+ MB


In [17]:
# Identify returns where Quantity is less than or equal to zero
returns = df1[df1['Quantity'] <= 0]
returns.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9839 entries, 178 to 525282
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      9839 non-null   object        
 1   StockCode    9839 non-null   object        
 2   Description  9839 non-null   object        
 3   Quantity     9839 non-null   int64         
 4   InvoiceDate  9839 non-null   datetime64[ns]
 5   Price        9839 non-null   float64       
 6   Customer ID  9839 non-null   float64       
 7   Country      9839 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 691.8+ KB


In [18]:
# Dropping row with less than or equal to zero quantity
df2 = df1[df1['Quantity'] > 0].copy()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407695 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      407695 non-null  object        
 1   StockCode    407695 non-null  object        
 2   Description  407695 non-null  object        
 3   Quantity     407695 non-null  int64         
 4   InvoiceDate  407695 non-null  datetime64[ns]
 5   Price        407695 non-null  float64       
 6   Customer ID  407695 non-null  float64       
 7   Country      407695 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 28.0+ MB


## Feature Engineering
 - Codense the data rows into a single summary for each customer
 - Create a new column: TotalPrice. Which calculates simply Quantity * Price for each row.
 - Calculate RFM metrics and create columns for them
 - Based on the RFM metrics rank the customers from 1-5. (1 being the worst and 5 being the best)

In [19]:
# Creating a new column 'TotalPrice' as the product of 'Quantity' and 'Price'
df2['TotalPrice'] = df2['Quantity'] * df2['Price']

# Setting the reference date for recency calculation
import datetime as dt
reference_date = df2['InvoiceDate'].max() + dt.timedelta(days=1)

# Calculating the RFM metrics (recency, frequency, monetary)
rfm = df2.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'Invoice': 'nunique',
    'TotalPrice': 'sum'
})
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.reset_index(inplace=True)
rfm.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4314 entries, 0 to 4313
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Customer ID  4314 non-null   float64
 1   Recency      4314 non-null   int64  
 2   Frequency    4314 non-null   int64  
 3   Monetary     4314 non-null   float64
dtypes: float64(2), int64(2)
memory usage: 134.9 KB


In [20]:
# Testing for a specific customer ID
customer12346 = rfm[rfm['Customer ID'] == 12346]

customer12346

,Customer ID,Recency,Frequency,Monetary
0,12346.0,165,11,372.86


In [21]:
# Making a score for each customer based on recency, frequency, and monetary value (1-5) 1 being worst and 5 being best

# Recency score
rfm['recency_score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

# Frequency score
rfm['frequency_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

# Monetary score
rfm['monetary_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm.head()

,Customer ID,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score
0,12346.0,165,11,372.86,2,5,2
1,12347.0,3,2,1323.32,5,2,4
2,12348.0,74,1,222.16,2,1,1
3,12349.0,43,3,2671.14,3,3,5
4,12351.0,11,1,300.93,5,1,2


In [22]:
# Creating segments
rfm['RFM_Segment'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)


# Segment mapping
segment_map = {
    r'5[4-5]': 'Most Valuable Customers', # Best Recency and Frequency
    r'[4-5][2-3]': 'Potential Loyal Customers', # Good Recency, Medium Frequency,
    r'51': 'New Customers', # Best Recency, Low Frequency,
    r'41': 'Promising Customers', # Good Recency, Low Frequency,
    r'[3-4][4-5]': 'Loyal Customers', # Medium Recency, Best Frequency,
    r'[2-3][1-3]': 'Need Attention', # Medium Recency and Frequency,
    r'[1-2][3-5]': 'At Risk', # Poor Recency, Best Frequency,
    r'[1-2][1-2]': 'Inactive' # Worst Recency and Frequency
}

# Implement the segment mapping
rfm['Segment'] = rfm['RFM_Segment'].replace(segment_map, regex=True)
rfm['Segment'].value_counts()

Segment
Need Attention               1149
Loyal Customers               742
Most Valuable Customers       663
Inactive                      643
Potential Loyal Customers     517
At Risk                       463
Promising Customers            87
New Customers                  50
Name: count, dtype: int64

## Saving the Findings

In [23]:
rfm.reset_index(inplace=True)

rfm.to_csv('../data/processed/rfm_segments_cleaned.csv', index=False)

print("RFM segmentation completed and saved to '../data/processed/rfm_segments_cleaned.csv'")

RFM segmentation completed and saved to '../data/processed/rfm_segments_cleaned.csv'
